In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt 

import tensorflow_addons as tfa
from pix2pix_Temporal_predictor import Generator

import datetime
import sys
import numpy as np
import os
import cv2
from glob import glob
x = 0

import argparse

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128, 128, 64) 3072        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64, 64, 128)  131584      sequential[0][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 32, 32, 256)  525312      sequential_1[0][0]               
______________________________________________________________________________________________

# class eval

In [2]:
class DataLoader():
    def __init__(self, dataset_name="default", img_res=(256, 256)):
        self.dataset_name = dataset_name
        self.img_res = img_res
        
    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('/datasets/%s/%s/*' % (self.dataset_name, data_type))#/content/drive/My Drive
        

        batch_images = np.random.choice(path, size=batch_size)
        #print(path)
        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('/datasets/%s/%sA/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        path_B = glob('/datasets/%s/%sB/*' % (self.dataset_name, data_type))
        #print(path_A)
        
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)
                '''

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)
                '''
                img_A = cv2.resize(img_A, self.img_res, interpolation = cv2.INTER_AREA)
                img_B = cv2.resize(img_B, self.img_res, interpolation = cv2.INTER_AREA)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def imread(self, path):
        return cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float)
      

In [3]:
class DataLoader(DataLoader):
    global x
    def load_data_seq(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))#content/drive/My Drive/cycleGAN/
        

        batch_images = np.random.choice(path, size=batch_size)
        #print(path)
        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

In [4]:
class DataLoader(DataLoader):
    def load_batch_seq(self, batch_size=1, is_testing=False):
        global x
        x=batch_size + x
        print('\n',x,'\n')
        data_type = "train" if not is_testing else "val"
        path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        #for i in range(n_batches):
        ba_A = path_A[x - batch_size : x]
        ba_B = path_B[x - batch_size : x]
            
            
        #path_A = np.random.choice(path_A, total_samples, replace=False)
        #path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)
                '''

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)
                '''
                img_A = cv2.resize(img_A, self.img_res, interpolation = cv2.INTER_AREA)
                img_B = cv2.resize(img_B, self.img_res, interpolation = cv2.INTER_AREA)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [5]:
class CycleGAN():
    def __init__(self):
        
        #sess and model of flownet2
        self.img_cols=256
        self.img_rows=256
        self.channels = 3 
        #flownet2 = initflow()
        epsilon: int = 1e-5
        self.img_shape = (self.img_rows, self.img_cols,self.channels) 
        self.dataset_name = '01'
        
        self.data_loader=DataLoader(self.dataset_name)
        #self.mc=myclass()
        
        
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1) 
        self.gf = 32                                                    
        self.df = 64   
        
        """
        hyperparameters
         1, lambda_cycle — controls howstrictly the cycle-consistency loss is enforced.
         2, lambda_id — influences identity loss
         3, lambda_opt — control optical flow difference between the two network
        """
        self.lambda_cycle = 10.0  
        self.lambda_id = 0.9 * self.lambda_cycle 
        self.lambda_opt = 10.0 
        
        optimizer = Adam(0.0002, 0.5)
        
        self.Pa = Generator()
        self.pb = Generator()
        
        #self.Pa.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        #self.Pb.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        
        self.d_A = self.build_discriminator('d_A')
        self.d_A.summary()
        
        self.d_B = self.build_discriminator('d_B')
        self.d_B.summary()
        
        self.d_A.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        self.d_B.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        
        self.g_AB = self.build_generator('g_AB')
        self.g_AB.summary()
        self.g_BA = self.build_generator('g_BA')
        self.g_BA.summary()
        
        img_A = tf.keras.Input(shape=self.img_shape)
        opt_A = tf.keras.Input(shape=self.img_shape)
        img_B = tf.keras.Input(shape=self.img_shape)
        opt_B = tf.keras.Input(shape=self.img_shape)
        
        print('shape of me is: ',img_A.shape[0])
        
        fake_B = self.g_AB(img_A)                                  
        fake_A = self.g_BA(img_B)
        #fake_B =myclass.flowwarp(fake_A,img_A)
        #opt_A = exect(flownet2,[img_A,img_A])
        #opt_B = img_B[0] 
            
        reconstr_A = self.g_BA(fake_B)                             
        reconstr_B = self.g_AB(fake_A)
        
        '''
        opt_A = exect(self.flownet2,[op1,op1])         
        opt_B = exect(self.flownet2,[self.prevB,op2])
        opt_Are = exect(self.flownet2,[self.prevAre,reconstr_A])
        opt_Bre = exect(self.flownet2,[self.prevBre,reconstr_B])
        '''
        
        self.prevA = img_A
        self.prevB = img_B
        
        self.prevAre = reconstr_A
        self.prevBre = reconstr_B
        
        img_A_id = self.g_BA(img_A)                                
        img_B_id = self.g_AB(img_B)
        
        self.d_A.trainable = False                                 
        self.d_B.trainable = False
        
        valid_A = self.d_A(fake_A)                                 
        valid_B = self.d_B(fake_B)
        
        
        
        self.combined = tf.keras.Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B,
                                       reconstr_A, reconstr_B,    
                                       img_A_id, img_B_id]) 
        self.combined.compile(loss=['mse', 'mse','mae', 'mae','mae', 'mae'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle, self.lambda_id, 
                                            self.lambda_id],
                              optimizer=optimizer)
        self.combined.summary()    

In [13]:
class CycleGAN(CycleGAN):
    def build_generator(self,name):
        d0 = tf.keras.Input(self.img_shape)  
        ''' ConvNET '''
        d1 = self.conv2d(d0, 64, 7)              
        d2 = self.conv2d(d1, 128, 3, 2, Ins_Norm=True)          
        d3 = self.conv2d(d2, 256, 3, 2, Ins_Norm=True)
        
        ''' Residual Block '''
        r1 = self.conv2d(d3, 256, 3, 1)
        r2 = self.conv2d(r1, 256, 3, 1)
        r3 = self.conv2d(r2, 256, 3, 1)
        r4 = self.conv2d(r3, 256, 3, 1)
        r5 = self.conv2d(r4, 256, 3, 1)
        
        r6 = self.resNET(r5, r4, 256, 3, 1)
        r7 = self.resNET(r6, r3, 256, 3, 1)
        r8 = self.resNET(r7, r2, 256, 3, 1)
        r9 = self.resNET(r8, r1, 256, 3, 1)
        
        u1 = self.deconv2d(r9, 128, 3, 2, True)
        u2 = self.deconv2d(u1, 64, 3, 2, True)
                        
        output_img = layers.Conv2D(self.channels, kernel_size=7, strides=(1,1), padding='same',activation='tanh')(u2) 
        x=tf.keras.Model(d0, output_img,name=name)
        #x.Summary()
        return x
    
    def build_discriminator(self,n):
        #conv2d(layer_input, num_Filter, f_size=4, Strides=1,Ins_Norm=True,Padding='same'):
        img=tf.keras.Input(self.img_shape)
        d1 = self.conv2d(img, 64, 4, 2)
        d2 = self.conv2d(d1, 128, 4, 2)
        d3 = self.conv2d(d2, 256, 4, 2)
        d4 = self.conv2d(d3, 512, 4, 2)  
        d5 = self.conv2d(d4, 1, 4, 1)  
       
        x=tf.keras.Model(img,d5,name=n)
        
        return x

In [14]:
class CycleGAN(CycleGAN):
    def sample_images(self, epoch, batch_i):
        r, c = 2, 3
        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)    
        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        #fake_A = self.g_BA.predict(imgs_B)
        '''
        cheeck here
        '''
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                #print(gen_imgs[cnt].shape)
                b,g,r = cv2.split(gen_imgs[cnt])       # get b,g,r
                im_rgb = cv2.merge([r,g,b])     # switch it to rgb
                
                axs[i,j].imshow((im_rgb * 255).astype(np.uint8))
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('on')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        
        plt.show()
        

In [15]:
class CycleGAN(CycleGAN):
    def train(self, epochs, batch_size=1, sample_interval=50):
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        for epoch in range(epochs):
            
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch_seq(batch_size)):
               
                # ----------------------
                #  Train Discriminators
                # ----------------------
                # Translate images to opposite domain
                
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                print(fake_A.shape,imgs_A.shape)
                
                ###################
                fake_A = floweval(self.sess,self.out,fake_A[0],fake_A[0],self.tens1,self.tens2)
                ###################
                
                #imagewarp()
                
                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
                # Total discriminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)
                
                # ------------------
                #  Train Generators
                # ------------------
                #print(d_loss,dA_loss,dB_loss)
                # Train the generators
                print(self.combined.metrics_names)
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                      [valid, valid,
                                                       imgs_A, imgs_B,
                                                       imgs_A, imgs_B])
                
                print(g_loss)
                
                #print('g_loss: ',g_loss)
                # If at save interval => plot the generated image samples
                
                if batch_i % sample_interval * 50 == 0:
                    self.sample_images(epoch, batch_i)

In [16]:
cycle_gan = CycleGAN()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Restoring parameters from ./models/LiteFlowNet2_Sintel_model
Model: "d_A"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 128, 128, 64)      3136      
_________________________________________________________________
instance_normalization (Inst (None, 128, 128, 64)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_1 (In (None, 6

Model: "g_BA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 256, 256, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
instance_normalization_24 (Inst (None, 256, 256, 64) 128         conv2d_121[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_20 (LeakyReLU)      (None, 256, 256, 64) 0           instance_normalization_24[0][0]  
_______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
g_BA (Model)                    (None, 256, 256, 3)  8137731     input_7[0][0]                    
                                                                 g_AB[1][0]                       
                                                                 input_5[0][0]                    
______________________________________________________________________________________________

In [ ]:
cycle_gan.train(epochs=10, batch_size=1, sample_interval=2)


 1 

(1, 256, 256, 3) (1, 256, 256, 3)
